In [1]:
from skimage.draw import circle
import numpy as np
import torch
import torch.nn.functional as F
margin = 5
img = np.zeros((margin, margin), dtype=np.uint8)

rr, cc = circle(margin/2, margin/2, margin/2+1, shape=(margin, margin))
img[rr, cc] = 1
fixed_depth = 6
#img = np.repeat(np.expand_dims(np.expand_dims(img, 0), 0), fixed_depth, 0) #[6, 1, 10, 10]
print(img, img.shape)

[[0 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]] (5, 5)


get labels

In [2]:
label = np.array([[1]*10, [2]*10, [3]*10,[4]*10, [5]*10, [6]*10, [7]*10, [8]*10, [9]*10, [0]*10], dtype=np.int32)
#background
back = np.zeros((fixed_depth, label.shape[0], label.shape[1]))
#print(label, label.shape, back, back.shape) #[6, 10, 10]

object_list = []

bincount = np.bincount(label.flatten())  # count number of occurences of each value non-negative ints
print(bincount)
min_fragment = 5
pixels = np.where(bincount > min_fragment)[0]  # M^K (k, 1)
print(pixels)
if len(pixels) > fixed_depth:
    pixels = pixels[:fixed_depth]
    print("Not all objects fits in fixed depth", RuntimeWarning)
#
for l, v in enumerate(pixels):
    back[l, label == v] = 1.
    object_list.append(np.array(range(l + 1)))
print(back[0], object_list) #object id in the original



[10 10 10 10 10 10 10 10 10 10]
[0 1 2 3 4 5 6 7 8 9]
Not all objects fits in fixed depth <class 'RuntimeWarning'>
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]] [array([0]), array([0, 1]), array([0, 1, 2]), array([0, 1, 2, 3]), array([0, 1, 2, 3, 4]), array([0, 1, 2, 3, 4, 5])]


In [3]:
sel = torch.from_numpy(img).float()

labels = torch.from_numpy(back).float()
labels = torch.unsqueeze(labels, dim=0)

sel = torch.unsqueeze(torch.stack([sel]*fixed_depth, dim=0), dim=1) #

print(labels.shape, sel.shape) #[1, 6, 10, 10] 6 is in channel , [6, 1, 10, 10], 6 is out channel
#exit(0)
#
print(labels.data[0,0], sel.data[0,0])
# labels = torch.from_numpy(back).float().to(device)
masks = F.conv2d(labels, sel, groups=fixed_depth, padding=margin // 2)
print(masks.data[0,0], masks.shape) #[1, 6, 5, 5]

# with the convolution, it get how many pixels are within the range

# after convolution we get
#


torch.Size([1, 6, 10, 10]) torch.Size([6, 1, 5, 5])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]) tensor([[0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0

### Dont know why we need to set the background channel to all 1s

In [4]:
masks[masks > 0] = 1. # for halos, has a weight of 1
masks[labels > 0] = 2. # for labels has a weight of 2
#masks[:, 0, :, :] = 1. # this is the background channel, all 1s, why??????

In [5]:
print(masks[0,0,:,:]) #backgroumd
masks[0, 1, :, :] #instance one

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]])


tensor([[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [6]:
weights=masks.sum(-1,keepdim=True).sum(-2,keepdim=True)


In [7]:
weights.shape, weights

(torch.Size([1, 6, 1, 1]), tensor([[[[40.]],
 
          [[40.]],
 
          [[50.]],
 
          [[60.]],
 
          [[60.]],
 
          [[60.]]]]))

In [8]:
masks = masks/weights
masks

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.0250, 0.0250, 0.0250, 0.0250, 0.0250, 0.0250, 0.0250, 0.0250,
           0.0250, 0.0250],
          [0.0250, 0.0250, 0.0250, 0.0250, 0.0250, 0.0250, 0.0250, 0.0250,
           0.0250, 0.0250],
          [0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500

# Halo Loss